# Limpiamos todos los csvs desde el 2010

In [ ]:
import pandas as pd
import numpy as np

## Llamamos a nuestros csvs complementarios

In [ ]:
df_educ_sup_tec = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/archivos_csv_complementos/cod_educ_sup_tec.csv')
df_cod_inst_edu = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/archivos_csv_complementos/cod_inst_educ.csv')
df_cod_departamentos = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/archivos_csv_complementos/cod_departamentos.csv')
df_cod_dominio_g = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/archivos_csv_complementos/cod_dominio_g.csv')
df_grad_estudios = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/archivos_csv_complementos/cod_grad_estudios.csv')
df_cod_idiomas = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/archivos_csv_complementos/cod_idioma.csv')
df_cod_estado_civil = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/archivos_csv_complementos/cod_estado_civil.csv')

## Creamos nuestras funciones

In [ ]:
def fun_merge(df1, df2, campo):
    return pd.merge(df1,
                    df2,
                    on = [campo],
                    how = 'left')

In [ ]:
def limpieza_csv_2018_2021(df, name_file_saved):
    df = df[['A�O', 'MES', 'CODPERSO', 'UBIGEO', 'DOMINIO', 'P207', 'P208A', 'P209', 'P300A', 'P301A', 'P301A1', 'P301B1', 'P301B3']]
    df.rename(columns = {'A�O': 'ANIO', 'DOMINIO': 'DOMINIO_G', 'P300A': 'COD_IDIOMA', 'P207': 'ID_SEXO', 'P208A': 'EDAD',
                'P209': 'ID_ESTADO_CIVIL', 'P301A': 'COD_GRADO_ESTUDIOS', 'P301A1':'COD_CARRERA_UNI',
                'P301B1': 'COD_CENTRO_ESTUDIOS', 'P301B3': 'COD_DEPARTAMENTO'}, inplace = True)
    df_clean = df.replace([' ', np.nan], -1)
    df_clean[['ID_ESTADO_CIVIL', 'COD_IDIOMA', 'COD_GRADO_ESTUDIOS', 'COD_CARRERA_UNI', 'COD_CENTRO_ESTUDIOS', 'COD_DEPARTAMENTO']] = df_clean[['ID_ESTADO_CIVIL', 'COD_IDIOMA', 'COD_GRADO_ESTUDIOS', 'COD_CARRERA_UNI', 'COD_CENTRO_ESTUDIOS', 'COD_DEPARTAMENTO']].astype(int)
    # df_sup_uni = df_clean[df_clean['COD_GRADO_ESTUDIOS']>=9]
    df_sup_uni = df_clean[(df_clean['COD_GRADO_ESTUDIOS']>=9) & (df_clean['COD_GRADO_ESTUDIOS']<=11)]
    df_sup_all = fun_merge(df_sup_uni, df_cod_dominio_g, "DOMINIO_G")
    df_sup_all = fun_merge(df_sup_all, df_cod_estado_civil, "ID_ESTADO_CIVIL")
    df_sup_all = fun_merge(df_sup_all, df_cod_idiomas, "COD_IDIOMA")
    df_sup_all = fun_merge(df_sup_all, df_grad_estudios, "COD_GRADO_ESTUDIOS")
    df_sup_all = fun_merge(df_sup_all, df_educ_sup_tec, "COD_CARRERA_UNI")
    df_sup_all = fun_merge(df_sup_all, df_cod_inst_edu, "COD_CENTRO_ESTUDIOS")
    df_sup_all = fun_merge(df_sup_all, df_cod_departamentos, "COD_DEPARTAMENTO")
    df_sup_all.drop(columns = ['MES', 'CODPERSO'], axis = 1, inplace = True)
    df_sup_all['SEXO'] = df_sup_all['ID_SEXO'].apply(lambda x: 'Hombre' if x == 1 else 'Mujer')
    df_sup_all.replace('Missing value', np.nan, inplace = True)
    df_sup_all.to_csv(f'/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA-LIMPIA/{name_file_saved}.csv', index = False)
    print('Guardado!')


In [ ]:
def limpieza_csv_2010_2017(df, name_file_saved):

    # arreglo para sacar solo los que son superiores a PREGRADO
    arr_g_a_sup_pregrado = ['superior universitaria incompleta', 'sup. univ. incompleta', 'Superior Universitaria Incompleta', 
    'superior universitaria completa', 'sup. univ. completa', 'Superior Universitaria Completa',
    'post-grado universitario', 'postgrado', 'Maestria/doctorado']

    # "['p301b1', 'p301b3'] not in index" = COD_CENTRO_ESTUDIOS, COD_DEPARTAMENTO
    if(int(name_file_saved) <= 2013):
        print('Entro menores 2013')
        df = df[['aÑo', 'dominio', 'p300a', 'p207', 'p208a', 'p209', 'p301a', 'p301a1']]
        df.rename(columns = {'aÑo': 'ANIO', 'dominio': 'DOMINIO_G', 'p300a': 'COD_IDIOMA', 'p207': 'ID_SEXO',
                'p208a': 'EDAD', 'p209': 'ID_ESTADO_CIVIL', 'p301a': 'COD_GRADO_ESTUDIOS',
                'p301a1':'COD_CARRERA_UNI'}, inplace = True)
        df_clean = df.replace([' ', '.', np.nan], -1)
        df_clean[['ID_ESTADO_CIVIL', 'COD_IDIOMA', 'COD_GRADO_ESTUDIOS', 'COD_CARRERA_UNI']] = df_clean[['ID_ESTADO_CIVIL', 'COD_IDIOMA', 'COD_GRADO_ESTUDIOS', 'COD_CARRERA_UNI']].astype(int)
        df_sup_uni = df_clean[(df_clean['COD_GRADO_ESTUDIOS']>=9) & (df_clean['COD_GRADO_ESTUDIOS']<=11)]
        df_sup_all = fun_merge(df_sup_uni, df_cod_dominio_g, "DOMINIO_G")
        df_sup_all = fun_merge(df_sup_all, df_cod_estado_civil, "ID_ESTADO_CIVIL")
        df_sup_all = fun_merge(df_sup_all, df_cod_idiomas, "COD_IDIOMA")
        df_sup_all = fun_merge(df_sup_all, df_grad_estudios, "COD_GRADO_ESTUDIOS")
        df_sup_all = fun_merge(df_sup_all, df_educ_sup_tec, "COD_CARRERA_UNI")
        df_sup_all['SEXO'] = df_sup_all['ID_SEXO'].apply(lambda x: 'Hombre' if x == 1 else 'Mujer')
        df_sup_all.replace('Missing value', np.nan, inplace = True)
        df_sup_all.to_csv(f'/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA-LIMPIA/{name_file_saved}.csv', index = False)
        print('Guardado!')

    # "['p301b3'] not in index" = COD_DEPARTAMENTO
    elif(int(name_file_saved) > 2013 and int(name_file_saved) <= 2015):
        print('Entro 2014 o 2015')
        df = df[['aÑo', 'dominio', 'p300a', 'p207', 'p208a', 'p209', 'p301a', 'p301a1', 'p301b1']]
        df.rename(columns = {'aÑo': 'ANIO', 'dominio': 'NOM_DOMINIO_G', 'p300a': 'NOM_IDIOMA', 'p207': 'SEXO',
                'p208a': 'EDAD', 'p209': 'NOM_ESTADO_CIVIL', 'p301a': 'NOM_GRADO_ESTUDIOS',
                'p301a1':'COD_CARRERA_UNI', 'p301b1': 'COD_CENTRO_ESTUDIOS'}, inplace = True)
        df_clean = df.replace([' ', np.nan], -1)
        df_clean[['COD_CARRERA_UNI', 'COD_CENTRO_ESTUDIOS']] = df_clean[['COD_CARRERA_UNI', 'COD_CENTRO_ESTUDIOS']].astype(int)
        df_sup_uni = df_clean[df_clean['NOM_GRADO_ESTUDIOS'].isin(arr_g_a_sup_pregrado)]
        df_sup_all = fun_merge(df_sup_uni, df_educ_sup_tec, "COD_CARRERA_UNI")
        df_sup_all = fun_merge(df_sup_all, df_cod_inst_edu, "COD_CENTRO_ESTUDIOS")
        df_sup_all.replace('Missing value', np.nan, inplace = True)
        df_sup_all.to_csv(f'/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA-LIMPIA/{name_file_saved}.csv', index = False)
        print('Guardado!')

    # años desde 2016 tienen otro formato
    else:
        print('Entro 2016 o 2017')
        df = df[['aÑo', 'dominio', 'p300a', 'p207', 'p208a', 'p209', 'p301a', 'p301a1', 'p301b1', 'p301b3']]
        df.rename(columns = {'aÑo': 'ANIO', 'dominio': 'NOM_DOMINIO_G', 'p300a': 'NOM_IDIOMA', 'p207': 'SEXO',
                    'p208a': 'EDAD',
                    'p209': 'NOM_ESTADO_CIVIL', 'p301a': 'NOM_GRADO_ESTUDIOS', 'p301a1':'COD_CARRERA_UNI',
                    'p301b1': 'COD_CENTRO_ESTUDIOS', 'p301b3': 'COD_DEPARTAMENTO'}, inplace = True)
        df_clean = df.replace([' ', np.nan], -1)
        df_clean[['COD_CARRERA_UNI', 'COD_CENTRO_ESTUDIOS', 'COD_DEPARTAMENTO']] = df_clean[['COD_CARRERA_UNI', 'COD_CENTRO_ESTUDIOS', 'COD_DEPARTAMENTO']].astype(int)
        df_sup_uni = df_clean[df_clean['NOM_GRADO_ESTUDIOS'].isin(arr_g_a_sup_pregrado)]
        df_sup_all = fun_merge(df_sup_uni, df_educ_sup_tec, "COD_CARRERA_UNI")
        df_sup_all = fun_merge(df_sup_all, df_cod_inst_edu, "COD_CENTRO_ESTUDIOS")
        df_sup_all = fun_merge(df_sup_all, df_cod_departamentos, "COD_DEPARTAMENTO")
        df_sup_all.replace('Missing value', np.nan, inplace = True)
        df_sup_all.to_csv(f'/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA-LIMPIA/{name_file_saved}.csv', index = False)
        print('Guardado!')

## Leemos nuestros datasets

In [ ]:
df_2010 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2010/enaho01a-2010-300.csv')
df_2011 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2011/enaho01a-2011-300.csv')
df_2012 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2012/enaho01a-2012-300.csv')
df_2013 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2013/enaho01a-2013-300.csv')
df_2014 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2014/enaho01a-2014-300.csv')
df_2015 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2015/enaho01a-2015-300.csv')

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3229: DtypeWarning: Columns (13,14,25,31,43,137,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3229: DtypeWarning: Columns (13,14,25,31,42,152,162) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3229: DtypeWarning: Columns (13,14,25,31,42,152,162,199,200,203,204) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3229: DtypeWarning: Columns (25,44,77,78,80

In [ ]:
df_2016 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2016/enaho01a-2016-300.csv')
df_2017 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2017/enaho01a-2017-300.csv')
df_2018 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2018/Enaho01a-2018-300.csv', sep = '|')
df_2019 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2019/Enaho01A-2019-300.csv', sep = '|')
df_2020 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2020/Enaho01A-2020-300.csv')
df_2021 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2021/Enaho01A-2021-300.csv')

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3229: DtypeWarning: Columns (27,78,79,81,82,84,86,87,88,89,90,92,94,96,98,100,104,107,110,111,112,114,115,116,117,118,119,120,121,122,124,216,217,219,220,230,232,233,235,270,271,272,273,274,297,301,457) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3229: DtypeWarning: Columns (27,78,84,86,87,89,92,96,98,100,104,106,107,110,111,112,114,115,116,117,118,119,120,121,122,123,124,217,218,219,220,230,232,233,234,270,271,272,273,274,318,322,478) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Limpieza de Información

In [ ]:
df_2021 = pd.read_csv('/work/enaho-personas-estudios-caracteristicas-data-science/ENAHO-DATA/ENAHO-2021/Enaho01A-2021-300.csv')

In [ ]:
df_2021.shape[0]

106580

In [ ]:
limpieza_csv_2018_2021(df_2021, '2021')

Guardado!


In [ ]:
limpieza_csv_2018_2021(df_2018, '2018')

Guardado!
/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
limpieza_csv_2018_2021(df_2019, '2019')

Guardado!
/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
limpieza_csv_2010_2017(df_2016, '2016')

/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
Guardado!


In [ ]:
limpieza_csv_2010_2017(df_2017, '2017')

Guardado!


In [ ]:
limpieza_csv_2010_2017(df_2010, '2010')

Entro menores 2013
/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
Guardado!


In [ ]:
# 2015 = "['p301b3'] not in index" = COD_DEPARTAMENTO
# 2014 = "['p301b3'] not in index" = COD_DEPARTAMENTO
# 2013 = "['p301b1', 'p301b3'] not in index" = COD_CENTRO_ESTUDIOS, COD_DEPARTAMENTO
# 2012 = "['p301b1', 'p301b3'] not in index" = COD_CENTRO_ESTUDIOS, COD_DEPARTAMENTO
# 2011 = "['p301b1', 'p301b3'] not in index" = COD_CENTRO_ESTUDIOS, COD_DEPARTAMENTO
# 2010 = "['p301b1', 'p301b3'] not in index" = COD_CENTRO_ESTUDIOS, COD_DEPARTAMENTO

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=61180446-cfea-4338-ac0e-81e50a732b69' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>